In [3]:
import pandas as pd
import torch

data = pd.read_csv('distillation-tower.csv')

X = data.drop('VapourPressure', axis=1)
X = X.iloc[:, 1:]
y = data['VapourPressure']

In [4]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [5]:
from torch.utils.data import TensorDataset, DataLoader

X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test.values, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [6]:
import torch.nn as nn

class VaporPressureNet(nn.Module):
    def __init__(self):
        super(VaporPressureNet, self).__init__()
        self.fc1 = nn.Linear(X.shape[1], 64) 
        self.fc2 = nn.Linear(64, 64)
        self.fc3 = nn.Linear(64, 1) 

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x


In [7]:
model = VaporPressureNet()

criterion = nn.MSELoss() 
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

epochs = 1000

for epoch in range(epochs):
    model.train() 
    running_loss = 0.0
    for inputs, targets in train_dataloader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs.squeeze(), targets)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)
    epoch_loss = running_loss / len(train_dataloader.dataset)
    print(f'Epoch {epoch+1}/{epochs}, Loss: {epoch_loss:.4f}')

model.eval() 
with torch.no_grad(): 
    predictions = []
    targets = []
    for inputs, true_values in test_dataloader:
        outputs = model(inputs)
        predictions.extend(outputs.squeeze().tolist())
        targets.extend(true_values.tolist())

from sklearn.metrics import r2_score

r2 = r2_score(targets, predictions)
print(f'R^2 Score on test set: {r2:.4f}')

Epoch 1/1000, Loss: 340.4881
Epoch 2/1000, Loss: 110.3719
Epoch 3/1000, Loss: 91.6977
Epoch 4/1000, Loss: 81.0839
Epoch 5/1000, Loss: 72.2547
Epoch 6/1000, Loss: 66.8820
Epoch 7/1000, Loss: 67.7764
Epoch 8/1000, Loss: 65.2304
Epoch 9/1000, Loss: 65.7007
Epoch 10/1000, Loss: 65.9817
Epoch 11/1000, Loss: 62.9534
Epoch 12/1000, Loss: 65.3266
Epoch 13/1000, Loss: 65.7273
Epoch 14/1000, Loss: 68.5311
Epoch 15/1000, Loss: 63.7260
Epoch 16/1000, Loss: 63.8254
Epoch 17/1000, Loss: 68.3848
Epoch 18/1000, Loss: 65.7079
Epoch 19/1000, Loss: 61.6535
Epoch 20/1000, Loss: 58.6006
Epoch 21/1000, Loss: 58.8058
Epoch 22/1000, Loss: 60.5219
Epoch 23/1000, Loss: 59.2326
Epoch 24/1000, Loss: 59.9195
Epoch 25/1000, Loss: 58.6585
Epoch 26/1000, Loss: 57.1460
Epoch 27/1000, Loss: 55.7289
Epoch 28/1000, Loss: 57.5245
Epoch 29/1000, Loss: 55.5259
Epoch 30/1000, Loss: 59.4773
Epoch 31/1000, Loss: 58.3643
Epoch 32/1000, Loss: 68.1867
Epoch 33/1000, Loss: 57.4430
Epoch 34/1000, Loss: 57.5437
Epoch 35/1000, Loss: 

In [8]:
def evaluate(n):
    single_observation = data.iloc[n, 1:-1]
    single_observation = single_observation.apply(pd.to_numeric, errors='coerce').fillna(0)
    single_observation_tensor = torch.tensor(single_observation.values, dtype=torch.float32).unsqueeze(0)
    model.eval()
    with torch.no_grad():
        prediction = model(single_observation_tensor)        
    print("Predicted Vapour Pressure:", prediction.item())
    print(y[n])